## **Introduction**

To recap what we have covered so far: In the **first notebook**, we introduced different methods for segmenting an image, such as using thresholds or the Simple Linear Iterative Clustering (SLIC) approach. In the **second notebook**, we continued by exploring how to extract useful attributes from each segment, known as features. These features include metrics like mean intensity per segment, standard deviation, rectangularity, compactness, and more.

In this notebook, we will bring everything together by using the features we've extracted to classify the segments we identified earlier. Specifically, our goal is to create a binary classifier that can determine whether each segment represents a tree in the image.

<img src="https://github.com/mariarodriguezn/obia_tutorials/blob/main/assets/general_workflow.png?raw=true" width="700">




## **Setup**
Please run the cell below. After that, go to **Runtime -> Restart session**, and confirm. Once the session restarts, move on to the next cell **without running this one again**.

In [ ]:
# Check if running on Google Colab
if 'google.colab' in str(get_ipython()):
    import os
    repo_dir = "obia_tutorials"
    marker_file = os.path.join(repo_dir, ".setup_done")

    # Setup the environment only if it hasn't been done already
    if not os.path.exists(marker_file):
        # Clone the repository
        !git clone https://github.com/mariarodriguezn/obia_tutorials.git

        # Install the required packages
        !pip install -r obia_tutorials/requirements.txt

        # Create a marker file to avoid re-running the setup
        with open(marker_file, 'w') as f:
            f.write("Setup completed")

In [ ]:
# imports
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import rasterio

from scipy import stats
from skimage.color import label2rgb
from skimage.measure import regionprops, regionprops_table, perimeter
from skimage.segmentation import mark_boundaries, slic
from skimage.util import map_array
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm


## **Image Segmentation**

We will continue working with the same high-resolution image. As a reminder, this image contains four spectral bands: Red, Green, Blue, and Near-Infrared (NIR). Additionally, we will utilize the Normalized Difference Vegetation Index (NDVI) that was calculated from these bands.

For the segmentation process we will again use SLIC algorithm with a compactness value of 0.3 and set the number of superpixels to 3000 as displayed below.

In [ ]:
# File path to the image
img_path = "obia_tutorials/sample_data/ortho_subset_I.tif"

# Read the image and extract the bands
with rasterio.open(img_path) as src:
    # Extract red, green, blue, and NIR bands, normalize to [0, 1]
    bands = src.read([1, 2, 3, 4]).astype(float) / 255
    red, green, blue, nir = bands

# Stack RGB into an array
rgb = np.stack([red, green, blue], axis=-1)

# Calculate NDVI
ndvi = (nir - red) / (nir + red + 1e-6)  # small value added to avoid division by zero

# Stack all bands into an array
image = np.stack([red, green, blue, nir], axis=-1)

# Set the fixed compactness and number of superpixels
compactness = 0.3
n_segments = 3000

# Generate segments using SLIC
segments = slic(image, n_segments=n_segments, compactness=compactness, start_label=1)

# Create the figure with 3 subplots
fig, axs = plt.subplots(ncols=3, figsize=(15, 5), constrained_layout=True)

# Display the original RGB image
axs[0].imshow(rgb)
axs[0].set_title("Original RGB")

# Display the RGB image with segmentation boundaries for 3000 segments
axs[1].imshow(mark_boundaries(rgb, segments, color=(1, 0, 0), mode="thick"))
axs[1].set_title(f"SLIC Segmentation Boundaries ({n_segments} segments)")

# Display the mean RGB per segment
axs[2].imshow(label2rgb(segments, rgb, kind='avg'))
axs[2].set_title("Mean RGB per Segment")

# Remove axis for all subplots
for ax in axs:
    ax.set_axis_off()

# Display the combined figure
plt.show()

## **Features Extraction**

As explained in the second notebook, there are multiple features that can be extracted to characterize the segments using `region_props` function from the`skiimage` Python library. In this case, to train our classifier later, we will calculate the following features for each segment :

* **Spectral Features** across different bands (Red, Green, Blue, NIR, and NDVI)
  * Intensity Mean
  * Standard Deviation

* **Shape Features**
  * Rectangularity
  * Compactness
  * Solidity

* **Textural Features**
  * Entropy of NDVI


In [ ]:
# Stack all bands into an array
full_stack_arr = np.stack([red, green, blue, nir, ndvi], axis=-1)

# Define custom features
def std(regionmask, intensity_img):
    vals = intensity_img[regionmask]
    std = np.std(vals)
    return std

def rectangularity(regionmask):
    return regionmask.sum()/regionmask.size

def compactness(regionmask):
    return 4*np.pi*regionmask.sum()/(perimeter(regionmask)**2 + 1e-6)  # small value added to avoid division by zero

def entropy_ndvi(regionmask, intensity_img):
    vals = intensity_img[regionmask]
    arr = stats.relfreq(vals, 100, defaultreallimits=(-1,1))[0]
    return stats.entropy(arr)

# Calculate set of various features for each segment
def calc_all_feats(seg_arr, img_arr):
    spectral_feats = pd.DataFrame(
        regionprops_table(
            label_image = seg_arr,
            intensity_image = img_arr,
            properties = ["label", "intensity_mean"],
            extra_properties=(std,)
        )
    )
    shape_feats = pd.DataFrame(
        regionprops_table(
            label_image = seg_arr,
            properties = ["solidity"],
            extra_properties=(rectangularity, compactness)
        )
    )

    textural_feats = pd.DataFrame(
        regionprops_table(
            label_image = seg_arr,
            intensity_image =img_arr[:, :, -1],  # Use only the NDVI band
            properties = [],
            extra_properties=(entropy_ndvi,)
        )
    )

    all_feats = pd.concat([spectral_feats, shape_feats, textural_feats], axis=1)

    return all_feats

all_feats_df = calc_all_feats(segments, full_stack_arr)
display(all_feats_df)

## **Image Classification**


Now, using the 14 features calculated above, we we will create a binary classifier to identify the trees in the image using Random Forest Algorithm.

Random Forest is a machine learning algorithm that works by creating multiple decision trees during training. Each tree makes a prediction, and the Random Forest aggregates these predictions to determine the final output. This method helps improve accuracy and reduces the risk of overfitting.

The key stages of this process are:

<img src="https://github.com/mariarodriguezn/obia_tutorials/blob/main/assets/image_classification_workflow.png?raw=true" width="800">






### **Label Image Segments**

For the purpose of this exercise, the sample segments labeled as trees and non-trees are already provided in the following cell. Additionally, a new column called **`class`** is added to the existing DataFrame `all_feats_df`, which contains all the segments along with their corresponding features. This new column indicates the class, with **1 for trees, 0 for non-trees, and -1 for unlabeled segments**.

In [ ]:
# Samples for both classes: trees and non-trees
trees_idxs = [
    1204, 1186, 1134, 1229, 1329, 1201, 1140, 1218, 1351, 962, 830, 886, 926, 1373, 1405, 1517, 1547, 1461, 1526, 1406, 1358, 1460, 1556, 735, 833, 838, 802, 513, 616, 565, 438, 258, 375, 334, 284, 204, 250, 155, 183, 241, 344, 359, 412, 328, 424, 506, 468, 554, 459, 579, 560, 769, 890, 791, 798, 811, 771, 846, 1076, 784, 919, 915, 133, 246, 356, 405, 425, 474, 166, 158, 34, 200, 209, 245, 55, 2003, 1984, 1910, 1813, 1873, 1923, 2296, 2220, 2143, 2113, 2153, 2319, 2275, 2284, 2422, 2451, 2398, 2460, 2455, 2208, 2218, 2199, 2036, 1986, 1924
]

non_trees_idxs = [
    2647, 2653, 2717, 2784, 2110, 2061, 2122, 1988, 1943, 2018, 1861, 1896, 1909, 1900, 1829, 1758, 1719, 1703, 1794, 1812, 1749, 1581, 1580, 1522, 1451, 1438, 1479, 1408, 1511, 1631, 1869, 1811, 1978, 2104, 2187, 2299, 2400, 2732, 2642, 2567, 2348, 2369, 2401, 2281, 1850, 1117, 197, 1002, 872, 216, 432, 418, 357, 382, 419, 336, 295, 351, 460, 644, 708, 800, 902, 1005, 1063, 174, 824, 831, 780, 1291, 1412, 1537, 1497, 1570, 1605, 1491, 1595, 1652, 1768, 1807, 1887, 1898, 1968, 1573, 1540, 1536, 1439, 1388, 1694, 2640
]

# Initialize the 'class' column in all_feats_df to -1 (for unlabeled segments)
all_feats_df['class'] = -1

# Update the 'class' column for tree segments (label 1)
all_feats_df.loc[all_feats_df['label'].isin(trees_idxs), 'class'] = 1

# Update the 'class' column for non-tree segments (label 0)
all_feats_df.loc[all_feats_df['label'].isin(non_trees_idxs), 'class'] = 0

# Display the updated DataFrame
all_feats_df = all_feats_df.rename(columns={'label': 'indx'})
display(all_feats_df)


To better understand the location of the given labeled samples, they can be displayed displayed based on the class.

In [ ]:
# Map the labeled segments onto the original segmented image
mapped_samples = map_array(
    segments,
    np.array(all_feats_df["indx"]),
    np.array(all_feats_df["class"])
)

# Define the colors:
# [-1] = undefined segments (grey), [0] = non-trees (dark blue), [1] = trees (green)
background_c, others_c, trees_c = '#7f8282', '#00008b', '#00FF00'
binary_cmap = mcolors.ListedColormap([background_c, others_c, trees_c])

# Display the original image with segment boundaries
plt.figure(figsize=(12, 8))
plt.imshow(mark_boundaries(rgb, segments, (1, 0, 0), mode="outer"))

# Overlay the labeled segments with the defined colors
plt.imshow(mapped_samples, cmap=binary_cmap, alpha=0.5)

# Turn off the axis for a cleaner look
plt.axis("off")

# Show the plot
plt.show()

#####**Task 1**
Taking into account the image above and the knowledge you have acquired in this course, answer: **What criteria are considered when selecting or creating samples for training in a classification model?** Hint: *An example of a criteria that could be mentioned and discuss is **Spatial Distribution**.*

**You answer goes here:**

### **Train Random Forest Classifier**

In this stage, we start by selecting only the image segments that we have labeled as "tree" or "non-tree," ignoring any segments without labels. We then split the data into two parts: the features columns (which are the characteristics we measured from each segment) and the class column (which tells us if the segment is a tree or not).

Next, we train a Random Forest model, which as mentioned is a type of machine learning method that creates many decision trees to improve accuracy. We set it to create 100 trees and use the Out-of-Bag (OOB) score to check how well the model might work on new, unseen data. The OOB score is a built-in way to test the model's accuracy without needing a separate test set, making it a practical way to assess how well the model is likely to perform.

#####**Task 2**
Complete the code to train the Random Forest Classifier (clf) using the following parameters:

1.   n_estimators = 100
2.   obb_score = True
3.   random_state = 42

*Hint: You can use the* "**RandomForestClassifier**" *method.*


In [ ]:
# Filter out the labeled samples
labeled_segments_df = all_feats_df[all_feats_df['class'] != -1]

# Separate features (X) and class label (y)
X = labeled_segments_df.drop(columns=['indx', 'class'])
y = labeled_segments_df['class']

#### Your code here to Train the Random Forest Classifier ####
clf =

clf.fit(X, y)

# Evaluate the model using OOB score
print("OOB Score:", clf.oob_score_)

#####**Task 3**
Based on the obtained OOB score, answer:


*  **What does the OOB score indicate about the performance of your model?** Consider what the score reveals about the model’s ability to generalize to new data and how well it distinguishes between 'tree' and 'non-tree' segments.




**You answer goes here:**

### **Test the Classifier**

Now, to further evaluate the classifier, we apply the trained classifier to the **unlabeled** segments. This step is fundamental to determine how well the classifier can generalize beyond the labeled training data and make accurate predictions on new, unseen segments.

In [ ]:
# Filter out the unlabeled segments
unlabeled_segments_df = all_feats_df[all_feats_df['class'] == -1]

# Separate features (X) for the unlabeled segments
X = unlabeled_segments_df.drop(columns=['indx', 'class'])

# Predict the class labels for the unlabeled segments
predictions = clf.predict(X)

# Update the original dataframe with the predictions
all_feats_df.loc[all_feats_df['class'] == -1, 'class'] = predictions

### **Interpret the results**

Finally, to actually analyze the resulting prediction on the unlabeled segments, below the results are plot.

In [ ]:
#  Map the predictions onto the original segmented image
mapped_predictions = map_array(
    segments,
    np.array(all_feats_df["indx"]),
    np.array(all_feats_df["class"]))

# Create the figure with 3 subplots
fig, axs = plt.subplots(ncols=3, figsize=(15, 5), constrained_layout=True)

# Display the original RGB image
axs[0].imshow(rgb)
axs[0].set_title("Original RGB")

# Display the RGB image with segmentation boundaries for 3000 segments
axs[1].imshow(mark_boundaries(rgb, segments, color=(1, 0, 0), mode="thick"))
axs[1].set_title(f"SLIC Segmentation Boundaries")

# Display the prediction result
axs[2].imshow(mapped_predictions, cmap=binary_cmap, interpolation="nearest")
axs[2].set_title("Random Forest classifier prediction")

# Remove axis for all subplots
for ax in axs:
    ax.set_axis_off()

# Display the combined figure
plt.show()

#####**Task 4**
Considering the classification results shown in the image above, discuss the following:

1.   Was the resulting classification good enough?
2.   Could the limited number of training samples have affected the model's performance?
3. What improvements could be made to enhance the accuracy of the classification?

**You answer goes here:**

## **Conclusion & Further thoughts**

This tutorial provided a simple example of how an image can be segmented, its features extracted, and then classified using a random forest algorithm.

Future directions for exploration could include scaling up these experiments by applying machine learning models like random forests to larger and more diverse datasets. A key area of interest might be identifying the smallest, most computationally efficient set of features that still achieve high accuracy for specific tasks. This is particularly relevant in the context of the growing prominence of deep learning models. Although training deep learning models requires large amounts of data and can be time-consuming, their inference time (meaning the time it takes to apply the model to new, unseen data) is typically very fast.